In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import os.path
from mako.template import Template
from tqdm import tqdm
import dateutil.parser
import cv2 as cv

In [ ]:
NCFILE = "data.nc"
root = Dataset( NCFILE, "r", format="NETCDF4")

In [ ]:
def nc_file_info( filename, root ):
    S = ""
    S += "Dataset: %s\n"%filename
    S += "------------------------------------------------------\n"
    S += "/ Variables:\n"
    for v in root.variables:
        S += " - %s\n"%v
    S += "Attributes:\n"
    for a in root.ncattrs():
        S += " - %s = %s\n"%(a,root.getncattr(a))
    S += "Groups: \n"
    for g in root.groups:
        S += " %s\n"%root[g].path
        S += "  Variables:\n"
        for v in root[g].variables:
            S += "    - %s\n"%v
        S += "  ncattrs:\n"
        for a in root[g].ncattrs():
            S += " - %s = %s\n"%(a,root[g].getncattr(a))
    return S
               
print( nc_file_info( NCFILE, root )  )

In [ ]:
def filetobase64( filename ):
    import base64
    with open(filename,'rb') as f:
        
        fb64 = base64.b64encode( f.read() )
        return fb64
    

In [ ]:
try:
    I0 = cv.imdecode( root["/cam0images"][1], cv.IMREAD_GRAYSCALE )
    I0 = cv.pyrDown(I0)
    cv.imwrite("frame.jpg",I0)
except IndexError:
    print("NetCDF file has no image data")

In [ ]:
plt.rcParams.update({'font.size': 18})
nsamples = root["/Z"].shape[0]
gridsize = root["/Z"].shape[1:3]

halfgridsize_i = int( gridsize[0]/2)
halfgridsize_j = int( gridsize[1]/2)

valid_samples_i = range( halfgridsize_i-5, halfgridsize_i+6 )
valid_samples_j = range( halfgridsize_j-5, halfgridsize_j+6 )


In [ ]:
timeserie = root["/Z"][:,halfgridsize_i,halfgridsize_j] * 1E-3
timeserie = timeserie - np.mean(timeserie)
t = root["/time"]

def crossings_nonzero_pos2neg(data):
    pos = data > 0
    return (pos[:-1] & ~pos[1:]).nonzero()[0]

crossings = crossings_nonzero_pos2neg(timeserie)

dmins = []
dmaxs = []
for ii in range( np.size(crossings)-1 ):
    datarange = np.arange(crossings[ii], crossings[ii+1])
    data = timeserie[ datarange ]
    dmax = np.argmax(data)
    dmin = np.argmin(data)
    dmins.append( datarange[dmin] )
    dmaxs.append( datarange[dmax] )
    
waveheights = np.array(timeserie[dmaxs]) - np.array(timeserie[dmins])
q = np.quantile( waveheights, 2.0/3.0)
print("quantile: ", q)

highestthirdwaves = waveheights[ waveheights>q ]
H13 = np.mean(highestthirdwaves)
print("H1/3: ", H13)

plt.figure( figsize=(20,10))
plt.plot(t, timeserie )
plt.scatter( t[crossings], np.zeros_like(crossings), c="r")
plt.scatter( t[dmins], timeserie[dmins], c="b")
plt.scatter( t[dmaxs], timeserie[dmaxs], c="g")
plt.grid()
plt.title("Timeserie at grid center. %d waves"%np.size(waveheights))
plt.xlabel("Time (secs.)")
plt.ylabel("Height (m)")
plt.savefig("timeserie.png")


Zcube = np.array( root["/Z"][:,valid_samples_i,valid_samples_j] * 1E-3 )
Hs = 4.0*np.std( Zcube-np.mean(Zcube) )
print("Hs: ", Hs)

#plt.figure( figsize=(10,10) )
#plt.hist( waveheights )

In [ ]:
# Spectrum
import scipy.signal

dt = t[2]-t[1]
print("dt: ",dt)

f, S = scipy.signal.csd(timeserie, timeserie, 1.0/dt, nperseg=512 )
for ii in valid_samples_i:
    for jj in valid_samples_j:
        timeserie_neigh = root["/Z"][:,ii,jj] * 1E-3
        timeserie_neigh = timeserie_neigh - np.mean(timeserie_neigh)
        _, S_neig = scipy.signal.csd(timeserie_neigh, timeserie_neigh, 1.0/dt, nperseg=512 )
        S += S_neig
S = S / float( np.size(valid_samples_i)*np.size(valid_samples_j) + 1)

# Using fft
#Nt = np.size(timeserie)
#dur=Nt*dt # s
#df=1.0/dur
#z_fft=np.fft.fftshift(np.fft.fft(timeserie) );
#z_fft=z_fft/Nt;
#z_fft=np.abs(z_fft)**2/df;
#z_fft=z_fft*np.sqrt(8.0/3.0);
#freq=np.linspace(-0.5,0.5,Nt)*(1/dt)

plt.figure( figsize=(10,10) )
plt.loglog( f, S)
plt.xticks([1E-2,1E-1,1E0,1E1])
plt.grid(which='minor')
plt.ylabel("S (m^2 s)")
plt.xlabel("fa (Hz)")
plt.title("Spectrum (Welch method) averaged\n in a central 11x11 grid region")
plt.savefig("spectrum.png")

In [ ]:
# Compute Hs
dFreq = np.gradient( f )
m0 = np.sum( S*dFreq )
m1 = np.sum( f*S*dFreq )
Hm0 = 4.0 * np.sqrt( m0 )
print("Hm0: ", Hm0)

# Peak frequency
pp = f[np.argmax( S )]
print("Peak frequency (Hz): ", pp)

# Average Period Tm01
Tm01 = m0/m1
print("Tm01: ", Tm01)


In [ ]:
# 2D Spectrum

Z = root["/Z"][3,:,:]
N = Z.shape[0]
Nm = int( N/2 )
dy = (root["/Y_grid"][2,0] - root["/Y_grid"][1,0])/1000.0
dx = (root["/X_grid"][0,2] - root["/X_grid"][0,1])/1000.0
print(dx,dy)




# 2D directional spectrum using Welch method

In [ ]:
# Extract a central part of the Zcube
N = 130

sequence_length = np.size(timeserie)
segments = 10
Nt = int(sequence_length / segments)
seg_shift = int(Nt/2)

print(Nt)

Zcube_mr = int( root["/Z"].shape[1] / 2 )
Zcube_mc = int( root["/Z"].shape[2] / 2 )
r_start, r_end = Zcube_mr-int(N/2)-50, Zcube_mr+int(N/2)-50+1
c_start, c_end = Zcube_mc-int(N/2), Zcube_mc+int(N/2)+1 

Nx = r_end - r_start
print("Nx: ",Nx)
Ny = c_end - c_start
print("Ny: ",Ny)
print("Nt: ",Nt)

kx_max=(2.0*np.pi/dx)/2.0;
ky_max=(2.0*np.pi/dy)/2.0;
f_max= (1.0/dt)/2.0;
dkx=2*np.pi/(dx*np.floor(Nx/2.0)*2.0); 
dky=2*np.pi/(dy*np.floor(Ny/2.0)*2.0);
df =1.0/(dt*np.floor(Nt/2.0)*2.0);

print( kx_max, ky_max, f_max, dkx, dky, df)

assert( Nx%2 != 0)
assert( Ny%2 != 0)
assert( Nt%2 == 0)

kx=np.arange(-kx_max,kx_max+dkx,dkx)
ky=np.arange(-ky_max,ky_max+dky,dky)
if Nt%2==0:
    f=np.arange(-f_max, f_max, df);
else:
    f=np.arange(-f_max, f_max+df, df);

KX, KY = np.meshgrid( kx, ky)
dkx=kx[3]-kx[2];
dky=ky[3]-ky[2];
KXY=np.sqrt(KX**2+KY**2);
print(dkx, dky)

In [ ]:
hanningx = scipy.signal.windows.hann(KX.shape[0])
hanningy = scipy.signal.windows.hann(KX.shape[1])
hanningt = scipy.signal.windows.hann(Nt)

Win3Dhann = np.tile( np.expand_dims( hanningx, axis=-1) * hanningy, (Nt,1,1) ) *  np.tile( np.expand_dims( np.expand_dims( hanningt, axis=-1 ), axis=-1 ), (1, KX.shape[0], KX.shape[1]) )

#  window correction factors
wc2x = 1.0/np.mean(hanningx**2);
wc2y = 1.0/np.mean(hanningy**2);
wc2t = 1.0/np.mean(hanningt**2);
wc2xy  = wc2x *wc2y;
wc2xyt = wc2xy*wc2t;

print(Win3Dhann.shape)
print(KX.shape)

Zcube_small = np.array( root["/Z"][0:Nt, r_start:r_end, c_start:c_end ] )
Zcube_small -= np.mean(Zcube_small)
Zcube_w = Zcube_small * Win3Dhann

print(dt, dkx, dky, df)

In [ ]:
plt.figure( figsize=(10,10))
plt.imshow(Zcube_small[20,:,:])
plt.title("Original surface")

plt.figure( figsize=(10,10))
plt.plot( np.squeeze( Zcube_w[:,60,60] ) )
plt.title("Windowed timeserie")
plt.figure( figsize=(10,10))
plt.imshow(Zcube_w[20,:,:])
plt.title("Windowed surface")

In [ ]:
S_welch = np.zeros_like( Win3Dhann )
n_samples = 0
print("Computing 3D fft via Welch's method")
for ii in tqdm(range(segments*2)):
    #print("Welch sample %d/%d"%(ii+1,segments*2))
    Zcube_small = np.array( root["/Z"][(ii*seg_shift):(ii*seg_shift+Nt), r_start:r_end, c_start:c_end ] )
    if Zcube_small.shape[0] != Nt:
        break
        
    Zcube_w = (Zcube_small - np.mean(Zcube_small) ) * Win3Dhann
    
    S = np.fft.fftshift( np.fft.fftn( Zcube_w, norm="ortho" ) );
    S /= (S.shape[0]*S.shape[1]*S.shape[2])
    S = np.abs(S)**2 / (dkx*dky*df)
    #-----------------------------
    #%%%%% corrects for window
    #----------------------------
    #%% FABIEN
    S *= wc2xyt
    
    # Store
    S_welch += S    
    n_samples += 1
    
S_welch /= n_samples    
print("Done!")

In [ ]:
min_freq = 0.30
max_freq = 0.8
num_plots = 8

start_freq_ii = np.argmin( np.abs(f-min_freq) )
end_freq_ii = np.argmin( np.abs(f-max_freq) )
indices = np.round( np.linspace(start_freq_ii, end_freq_ii, num_plots ) ).astype(np.uint32)

for ii in indices:

    plt.figure( figsize=(11,10))    

    #dummy = np.flipud( 2* np.mean(S_welch[ mdt+ii-1:mdt+ii+2,:,:], axis=0) )    
    dummy = 2* np.mean(S_welch[ ii-1:ii+2,:,:], axis=0) 
    
    dummy_cen = np.copy(dummy)
    dummy_cen[ int(dummy_cen.shape[0]/2)-1:int(dummy_cen.shape[0]/2)+1, int(dummy_cen.shape[1]/2)-1:int(dummy_cen.shape[1]/2)+1 ] = 0
    maxidx = np.unravel_index( np.argmax(dummy_cen), dummy_cen.shape )
    
    qp=( np.arctan2( KY[ maxidx[0],maxidx[1] ], KX[ maxidx[0],maxidx[1] ]) )/np.pi*180.0
    if qp<0:
        qp=qp+360

    kp=np.sqrt( KX[ maxidx[0],maxidx[1] ]**2 + KY[ maxidx[0],maxidx[1] ]**2 );

    plt.pcolor(KX,KY, 10*np.log10(dummy) )
    plt.clim( 10*np.array([-4.0 + np.amax(np.log10(dummy)), -0+np.amax(np.log10(dummy))]) )
    plt.colorbar()

    plt.scatter( [KX[ maxidx[0],maxidx[1] ]], [KY[ maxidx[0],maxidx[1] ]], marker="x", s=100, c="k" )

    plt.ylim([-2.5,2.5])
    plt.xlim([-2.5,2.5])

    plt.xlabel("Kx (rad/m)")
    plt.ylabel("Ky (rad/m)")
    plt.title("S_kx_ky, fa=%3.2f (Hz).\n Peak angle: %3.0f°, mag: %2.3f (rad/m)\n"%( f[ii],qp,kp ) )
    


In [ ]:
_, ncfilef = os.path.split(NCFILE)
print(ncfilef)

In [ ]:
timestring = root["/meta"].getncattr("timestring")
print(timestring)
datadatetime = dateutil.parser.parse(timestring)
print(datadatetime.time())
print(datadatetime.date())

In [ ]:
mytemplate = Template(filename="analysis_template.html", strict_undefined=True)

T = mytemplate.render( title="%s wave analysis"%NCFILE,
                       framedata=filetobase64("frame.jpg").decode("ascii"),
                       timeseriedata=filetobase64("timeserie.png").decode("ascii"),
                       spectrumdata=filetobase64("spectrum.png").decode("ascii"),
                       dirspectrumdata=filetobase64("spectrum_dir.png").decode("ascii"),
                       hs="%2.3f"%Hs,
                       hm0="%2.3f"%Hm0,
                       date=datadatetime.date(),
                       time=datadatetime.time(),
                       ncfile="%s (%s)"%(ncfilef,root["/meta"].getncattr("datafile")),
                       duration="%d secs."%(t[-1]-t[0]),
                       fps="%3.1f"%(root["/meta"].getncattr("fps")),
                       meta=nc_file_info( NCFILE, root ).replace('\n','<br />'))

with open("analysis.html","w") as f:
    f.write(T)
    